In [1]:
import pandas as pd
import torch as torch
from transformers import BertModel
from pytorch_pretrained_bert import BertTokenizer

In [2]:
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
import statistics

In [5]:
import time

In [6]:
from functools import partial
# from dataclasses import dataclass
from collections import OrderedDict

In [7]:
import torch.nn as nn

In [8]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
bert_tokenizer=BertTokenizer.from_pretrained('bertPytorch/bert-base-cased')
embedding_model=BertModel.from_pretrained('bertPytorch/bert-base-cased')

In [11]:
embedding_model.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [12]:
def BertEmbedding(ids,masks):
#         input_token = ['[CLS]'] + bert_tokenizer.tokenize(text_as_input) + ['[SEP]']
#         input_id = bert_tokenizer.convert_tokens_to_ids(input_token)
#         input_id=[input_id]
#         input_ids = pad_sequences(input_id, maxlen=128, dtype="long", truncating="post", padding="post")
#         attention_masks = []
#         for seq in text_as_input_ids:
#             for i in seq:
#                 if i > 0:
#                     attention_masks.append(1)
#                 else:
#                     attention_masks.append(0)
#         masks=[attention_masks]
#         masks=torch.LongTensor(masks)
#         input_ids=torch.LongTensor(text_as_input_ids)
        embeddings = embedding_model(ids,masks)
        embeddings = embeddings[0]
        
        return embeddings

In [13]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False) 

In [14]:
conv = conv3x3(in_channels=32, out_channels=64)
print(conv)
del conv

Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [15]:
def activation_func(activation):
    return  nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
        ['selu', nn.SELU(inplace=True)],
        ['sigmoid',nn.Sigmoid()],
        ['none', nn.Identity()]
    ])[activation]

In [16]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels, self.out_channels, self.activation = in_channels, out_channels, activation
        self.blocks = nn.Identity()
        self.activate = activation_func(activation)
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.activate(x)
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

In [17]:
dummy = torch.ones((1, 1, 1, 1))

block = ResidualBlock(1, 64)
block(dummy)

tensor([[[[2.]]]])

In [18]:
class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(
            nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            nn.BatchNorm2d(self.expanded_channels)) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels


In [19]:
def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(conv(in_channels, out_channels, *args, **kwargs), nn.BatchNorm2d(out_channels))


In [20]:
class ResNetBasicBlock(ResNetResidualBlock):
    """
    Basic ResNet block composed by two layers of 3x3conv/batchnorm/activation
    """
    expansion = 1
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        )

In [21]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
           conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
             activation_func(self.activation),
             conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsampling),
             activation_func(self.activation),
             conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1),
        )

In [22]:
class ResNetLayer(nn.Module):
    """
    A ResNet layer composed by `n` blocks stacked one after the other
    """
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        )

    def forward(self, x):
        x = self.blocks(x)
        return x

In [23]:
class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by layers with increasing features.
    """
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[2,2,2,2], 
                 activation='relu', block=ResNetBasicBlock, *args, **kwargs):
        super().__init__()
        self.blocks_sizes = blocks_sizes
        
        self.gate = nn.Sequential(
            nn.Conv2d(in_channels, self.blocks_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.blocks_sizes[0]),
            activation_func(activation),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], activation=activation, 
                        block=block,*args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, activation=activation, 
                          block=block, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]       
        ])
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x

In [24]:
class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.Linear(in_features, n_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        x = self.sigmoid(x)
        return x


In [25]:
class ResNet(nn.Module):
    
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs)
        self.decoder = ResnetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [26]:
def resnet152(in_channels, n_classes, block=ResNetBottleNeckBlock, *args, **kwargs):
    return ResNet(in_channels, n_classes, block=block, deepths=[3, 8, 36, 3], *args, **kwargs)

In [27]:
model = resnet152(2, 1, activation='sigmoid')
print(model.cuda(), (3, 224, 224))

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Sigmoid()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBottleNeckBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): Sigmoid()
              (2): Sequential(
                (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [28]:
class FakeNewsClassifier(nn.Module):
    def __init__(self, resnet_architecture, embedding_func):
        super(FakeNewsClassifier, self).__init__()
        self.resnet=resnet_architecture
    def forward(self, x):
        output = self.resnet(x)
        return output

In [29]:
model = FakeNewsClassifier(resnet152(128, 1, activation='sigmoid'), BertEmbedding)

In [30]:
def getIds(inp):
    ids=[]
    masks=[]
    for sent in inp:
        input_token = ['[CLS]'] + bert_tokenizer.tokenize(sent) + ['[SEP]']
        input_id = bert_tokenizer.convert_tokens_to_ids(input_token)
        input_id=[input_id]
        input_ids = pad_sequences(input_id, maxlen=128, dtype="long", truncating="post", padding="post")
        attention_masks = []
        for seq in input_ids:
            for i in seq:
                if i > 0:
                    attention_masks.append(1)
                else:
                    attention_masks.append(0)
        maks=[attention_masks]
        masks.append(maks)
        ids.append(input_ids)
    return(ids,masks)

In [31]:
def getIds(inp):
        ids=[]
        masks=[]
        for sent in inp:
            input_token = ['[CLS]'] + bert_tokenizer.tokenize(sent) + ['[SEP]']
            input_id = bert_tokenizer.convert_tokens_to_ids(input_token)
            input_id=[input_id]
            input_ids = pad_sequences(input_id, maxlen=128, dtype="long", truncating="post", padding="post")
            attention_masks = []
            for seq in input_ids:
                for i in seq:
                    if i > 0:
                        attention_masks.append(1)
                    else:
                        attention_masks.append(0)
            maks=[attention_masks]
            masks.append(maks)
            ids.append(input_ids)
        return(ids,masks)

In [32]:
class DataGenerator:
    def __init__(self,data):
        MAX_LEN = 128
        batch_size=40
        data['split'].value_counts()
        trainDf=data[data['split']=='TRAIN']
        valDf=data[data['split']=='VALID']
        testDf=data[data['split']=='TEST']
        print(trainDf.head())
        print(valDf.head())
        print(testDf.head())

        
        trainText = trainDf['text'].values
        trainLabels = trainDf['truth'].values
        valText = valDf['text'].values
        valLabels = valDf['truth'].values
        testText = testDf['text'].values
        testLabels = testDf['truth'].values


        train_ids, train_attention_masks = getIds(trainText)
        val_ids, val_attention_masks = getIds(valText)
        test_ids, test_attention_masks = getIds(testText)

        train_data_inputs = torch.tensor(train_ids)
        train_data_labels = torch.tensor(trainLabels)
        train_data_masks = torch.tensor(train_attention_masks)
        val_data_inputs = torch.tensor(val_ids)
        val_data_labels = torch.tensor(valLabels)
        val_data_masks = torch.tensor(val_attention_masks)
        test_data_inputs = torch.tensor(test_ids)
        test_data_labels = torch.tensor(testLabels)
        test_data_masks = torch.tensor(test_attention_masks)

        trainDataset = TensorDataset(train_data_inputs, train_data_masks, train_data_labels)
        testDataset = TensorDataset(test_data_inputs, test_data_masks, test_data_labels)
        valDataset = TensorDataset(val_data_inputs, val_data_masks, val_data_labels)
        train_data_sampler=RandomSampler(trainDataset)
        test_data_sampler=RandomSampler(testDataset)
        val_data_sampler=RandomSampler(valDataset)

        self.train_dataloader = DataLoader(trainDataset, sampler=train_data_sampler, batch_size=batch_size)
        self.valid_dataloader = DataLoader(valDataset, sampler=test_data_sampler, batch_size=batch_size)
        self.test_dataloader = DataLoader(testDataset,sampler=test_data_sampler, batch_size=batch_size)

        
    def generate_train_batch_data(self):
        for i, batch in enumerate(self.train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            yield batch

    def generate_valid_batch_data(self):
        for i, batch in enumerate(self.valid_dataloader):
            batch = tuple(t.to(device) for t in batch)
            yield batch

    def generate_test_batch_data(self):
        for i, batch in enumerate(self.test_dataloader):
            batch = tuple(t.to(device) for t in batch)
            yield batch

In [33]:
class DataGeneratorv2:
    def __init__(self):
        MAX_LEN = 128
        batch_size=10
        data=pd.read_csv('/home/gridsan/svattam/upama/FakeNews/combined_relevant_data.csv')
        text=data[['text']]
        labels=data[['truth']].to_numpy()
        tokenizer= BertTokenizer.from_pretrained('/home/gridsan/svattam/bert/bert-base-cased', do_lower_case=True)
        x_train, X_test, Y_train, y_test = train_test_split(text,labels, test_size=0.3)
        x_val, x_test, Y_val, Y_test = train_test_split(X_test, y_test, test_size=0.5)
        
        train_inputs, train_attention_masks = prepare_for_bert(x_train, tokenizer, MAX_LEN)
        val_inputs, val_attention_masks = prepare_for_bert(x_val, tokenizer, MAX_LEN)
        test_inputs, test_attention_masks = prepare_for_bert(x_test, tokenizer, MAX_LEN)
        
        train_data_inputs = torch.tensor(train_inputs)
        train_data_labels = torch.tensor(Y_train)
        train_data_masks = torch.tensor(train_attention_masks)
        val_data_inputs = torch.tensor(val_inputs)
        val_data_labels = torch.tensor(Y_val)
        val_data_masks = torch.tensor(val_attention_masks)
        test_data_inputs = torch.tensor(test_inputs)
        test_data_labels = torch.tensor(Y_test)
        test_data_masks = torch.tensor(test_attention_masks)
       
        trainDataset = TensorDataset(train_data_inputs, train_data_masks, train_data_labels)
        testDataset = TensorDataset(test_data_inputs, test_data_masks, test_data_labels)
        valDataset = TensorDataset(val_data_inputs, val_data_masks, val_data_labels)
        train_data_sampler=RandomSampler(trainDataset)
        test_data_sampler=RandomSampler(testDataset)
        val_data_sampler=RandomSampler(valDataset)


        # self.train_data = 
        # self.train_masks = 
        # self.train_labels = 

        # self.valid_data = 
        # self.valid_masks = 
        # self.valid_labels = 

        self.train_dataloader = DataLoader(trainDataset, sampler=train_data_sampler, batch_size=batch_size)
        self.valid_dataloader = DataLoader(valDataset, sampler=test_data_sampler, batch_size=batch_size)
        self.test_dataloader = DataLoader(testDataset,sampler=test_data_sampler, batch_size=batch_size)

    # def sampleFromClass(ds, k):
    #     class_counts = {}
    #     train_data = []
    #     train_label = []
    #     test_data = []
    #     test_label = []
    #     for data, label in ds:
    #         c = label.item()
    #         class_counts[c] = class_counts.get(c, 0) + 1
    #         if class_counts[c] <= k:
    #             train_data.append(data)
    #             train_label.append(torch.unsqueeze(label, 0))
    #         else:
    #             test_data.append(data)
    #             test_label.append(torch.unsqueeze(label, 0))
    #     train_data = torch.cat(train_data)
    #     for ll in train_label:
    #         print(ll)
    #     train_label = torch.cat(train_label)
    #     test_data = torch.cat(test_data)
    #     test_label = torch.cat(test_label)

    #     return (TensorDataset(train_data, train_label), 
    #         TensorDataset(test_data, test_label))
        
    def generate_train_batch_data(self):
        for i, batch in enumerate(self.train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            yield batch

    def generate_valid_batch_data(self):
        for i, batch in enumerate(self.valid_dataloader):
            batch = tuple(t.to(device) for t in batch)
            yield batch

    def generate_test_batch_data(self):
        for i, batch in enumerate(self.test_dataloader):
            batch = tuple(t.to(device) for t in batch)
            yield batch


In [34]:
data=pd.read_csv('/home/gridsan/svattam/upama/FakeNews/combined_relevant_data.csv')

In [35]:
df = data.sample(frac=1).reset_index(drop=True)


In [36]:
sample_data=df.iloc[0:100]

In [37]:
def prepare_for_bert(df, tokenizer, MAX_LEN):
    sentences=df['text'].values
    sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return input_ids, attention_masks

In [38]:
data_gen=DataGeneratorv2()

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [40]:
class Embedder:
    def __init__(self):
        self.embedding_model = BertModel.from_pretrained('/home/gridsan/svattam/bert/bert-base-cased', from_tf=False)
        self.embedding_model.cuda()

In [41]:
embedder = Embedder()


In [42]:
model.cuda()

FakeNewsClassifier(
  (resnet): ResNet(
    (encoder): ResNetEncoder(
      (gate): Sequential(
        (0): Conv2d(128, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Sigmoid()
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      )
      (blocks): ModuleList(
        (0): ResNetLayer(
          (blocks): Sequential(
            (0): ResNetBottleNeckBlock(
              (blocks): Sequential(
                (0): Sequential(
                  (0): Conv2dAuto(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                )
                (1): Sigmoid()
                (2): Sequential(
                  (0): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (1): Bat

In [43]:
n_epochs = 10
batch_size = 40
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.BCELoss()
metrics=[]
for epoch in range(n_epochs):
    start_time = time.time()
    train_loss = []
    val_loss=[]
    train_acc=[]
    val_acc=[]
    for i, batch in enumerate(data_gen.generate_train_batch_data()):
        print(i)
        model.train(True)
        b_input_ids, b_input_mask, labels = batch
        print(b_input_ids.size(),b_input_mask.size())
        with torch.no_grad():
            embeddings = embedder.embedding_model(b_input_ids, attention_mask=b_input_mask)
        X_batch = embeddings[0].float()
        X_batch = X_batch.unsqueeze(3)
        y_batch = labels.float()
        y_pred = model(X_batch)
        print("=================")
        print(y_batch)
        print(y_pred)
        print("=================")
        exit()
        optimizer.zero_grad()
        loss = loss_fn(y_pred, y_batch)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        y_labels=y_pred.cpu().detach().numpy()
        cat=np.argmax(y_labels, axis=1)
        clabels=np.argmax(labels.cpu().detach().numpy(), axis=1)
        train_acc.append(accuracy_score(clabels,cat))
    for i, batch in enumerate(data_gen.generate_valid_batch_data()):
        print(i)
        correct=0
        model.eval()
        b_input_ids, b_input_mask, labels = batch
        with torch.no_grad():
            embeddings = embedder.embedding_model(b_input_ids, attention_mask=b_input_mask)
        X_batch = embeddings[0].float()
        y_batch = labels.float()
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        val_loss.append(loss.item())
        y_labels=y_pred.cpu().detach().numpy()
        cat=np.argmax(y_labels, axis=1)
        clabels=np.argmax(labels.cpu().detach().numpy(), axis=1)
        val_acc.append(accuracy_score(clabels,cat))   
    trainingAccuracy=statistics.mean(train_acc)
    validationAccuracy=statistics.mean(val_acc)
    trainingLoss=statistics.mean(train_loss)
    validationLoss=statistics.mean(val_loss)
    metrics.append([trainingAccuracy,trainingLoss,validationAccuracy,validationLoss])
print(metrics)
stats = np.array(metrics)
np.savez("trainingstats.npz", stats)


0
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.]], device='cuda:0')
tensor([[0.4786],
        [0.4720],
        [0.4882],
        [0.4708],
        [0.4657],
        [0.4797],
        [0.4828],
        [0.4851],
        [0.4909],
        [0.4871]], device='cuda:0', grad_fn=<SigmoidBackward>)
1
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]], device='cuda:0')
tensor([[0.7720],
        [0.7312],
        [0.8079],
        [0.7906],
        [0.7616],
        [0.7453],
        [0.7763],
        [0.7917],
        [0.7898],
        [0.7633]], device='cuda:0', grad_fn=<SigmoidBackward>)
2
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.]

18
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.]], device='cuda:0')
tensor([[0.7110],
        [0.7632],
        [0.7424],
        [0.6682],
        [0.6067],
        [0.7500],
        [0.7660],
        [0.7182],
        [0.4720],
        [0.7310]], device='cuda:0', grad_fn=<SigmoidBackward>)
19
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.]], device='cuda:0')
tensor([[0.6865],
        [0.4863],
        [0.6012],
        [0.6853],
        [0.6962],
        [0.8042],
        [0.6540],
        [0.5625],
        [0.7022],
        [0.6909]], device='cuda:0', grad_fn=<SigmoidBackward>)
20
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [

36
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.]], device='cuda:0')
tensor([[0.6759],
        [0.6660],
        [0.6727],
        [0.6888],
        [0.6810],
        [0.6623],
        [0.6612],
        [0.6668],
        [0.6957],
        [0.6583]], device='cuda:0', grad_fn=<SigmoidBackward>)
37
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.]], device='cuda:0')
tensor([[0.7507],
        [0.7556],
        [0.7566],
        [0.7465],
        [0.7522],
        [0.7570],
        [0.7549],
        [0.7532],
        [0.7493],
        [0.7449]], device='cuda:0', grad_fn=<SigmoidBackward>)
38
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [

54
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.]], device='cuda:0')
tensor([[0.8124],
        [0.7905],
        [0.8215],
        [0.8005],
        [0.8027],
        [0.8165],
        [0.7955],
        [0.7889],
        [0.8092],
        [0.8322]], device='cuda:0', grad_fn=<SigmoidBackward>)
55
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.]], device='cuda:0')
tensor([[0.8289],
        [0.8020],
        [0.8193],
        [0.8125],
        [0.8064],
        [0.8011],
        [0.8135],
        [0.8205],
        [0.8181],
        [0.8270]], device='cuda:0', grad_fn=<SigmoidBackward>)
56
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [

72
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.]], device='cuda:0')
tensor([[0.4738],
        [0.5193],
        [0.4552],
        [0.4947],
        [0.5066],
        [0.4711],
        [0.4675],
        [0.5290],
        [0.4517],
        [0.5241]], device='cuda:0', grad_fn=<SigmoidBackward>)
73
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0')
tensor([[0.5890],
        [0.5535],
        [0.5479],
        [0.5392],
        [0.5940],
        [0.5616],
        [0.5153],
        [0.5505],
        [0.5094],
        [0.5671]], device='cuda:0', grad_fn=<SigmoidBackward>)
74
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [

...

1080
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.]], device='cuda:0')
tensor([[0.8037],
        [0.7990],
        [0.8050],
        [0.8001],
        [0.8087],
        [0.8102],
        [0.8105],
        [0.8032],
        [0.8101],
        [0.8027]], device='cuda:0', grad_fn=<SigmoidBackward>)
1081
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]], device='cuda:0')
tensor([[0.7837],
        [0.7969],
        [0.7861],
        [0.7947],
        [0.7910],
        [0.7863],
        [0.7887],
        [0.7949],
        [0.7935],
        [0.7936]], device='cuda:0', grad_fn=<SigmoidBackward>)
1082
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
   

1098
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.]], device='cuda:0')
tensor([[0.6871],
        [0.6910],
        [0.6908],
        [0.6764],
        [0.6950],
        [0.6943],
        [0.6981],
        [0.6958],
        [0.6883],
        [0.6939]], device='cuda:0', grad_fn=<SigmoidBackward>)
1099
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.]], device='cuda:0')
tensor([[0.6179],
        [0.6152],
        [0.6097],
        [0.6097],
        [0.6019],
        [0.6038],
        [0.6038],
        [0.6025],
        [0.6147],
        [0.6129]], device='cuda:0', grad_fn=<SigmoidBackward>)
1100
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
   

1116
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.]], device='cuda:0')
tensor([[0.4281],
        [0.4325],
        [0.4318],
        [0.4332],
        [0.4377],
        [0.4441],
        [0.4268],
        [0.4456],
        [0.4319],
        [0.4295]], device='cuda:0', grad_fn=<SigmoidBackward>)
1117
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0')
tensor([[0.3840],
        [0.3847],
        [0.3861],
        [0.3837],
        [0.3873],
        [0.3877],
        [0.3875],
        [0.3875],
        [0.3825],
        [0.3931]], device='cuda:0', grad_fn=<SigmoidBackward>)
1118
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
   

1134
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.]], device='cuda:0')
tensor([[0.5344],
        [0.5610],
        [0.5507],
        [0.5419],
        [0.5470],
        [0.5298],
        [0.5594],
        [0.5388],
        [0.5513],
        [0.5234]], device='cuda:0', grad_fn=<SigmoidBackward>)
1135
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.]], device='cuda:0')
tensor([[0.5426],
        [0.5764],
        [0.5506],
        [0.5482],
        [0.5665],
        [0.5781],
        [0.5801],
        [0.5421],
        [0.5817],
        [0.5769]], device='cuda:0', grad_fn=<SigmoidBackward>)
1136
torch.Size([10, 128]) torch.Size([10, 128])
tensor([[1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
   

KeyboardInterrupt: 